In [43]:
import pandas as pd
from pandas import DataFrame
from os import listdir, chdir
from os.path import isfile, join
import math
import matplotlib.pyplot as plt

In [2]:
mypath = '/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity'
chdir(mypath)
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.endswith('norm.csv') and f[0].isdigit()]

In [3]:
onlyfiles

['2016_norm.csv',
 '2015_norm.csv',
 '2014_norm.csv',
 '2017_norm.csv',
 '2012_norm.csv',
 '2011_norm.csv',
 '2010_norm.csv',
 '2018_norm.csv',
 '2013_norm.csv']

In [4]:
dfs = []
for file in onlyfiles:
    dfs.append(pd.read_csv(file))
df = pd.concat(dfs).reset_index()

In [9]:
train_cols = ['max_temp', 'min_temp','mean_temp', 'res_max', 'dom_vel', 'rain_7days', 'dem', 'slope',
       'curvature', 'aspect', 'ndvi_new', 'evi', 'lst_day', 'lst_night','max_dew_temp', 'mean_dew_temp', 
       'min_dew_temp', 'dir_max_1','dir_max_2', 'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6',
       'dir_max_7', 'dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3','dom_dir_4', 'dom_dir_5', 'dom_dir_6', 
       'dom_dir_7', 'dom_dir_8','corine_111', 'corine_112', 'corine_121', 'corine_122', 'corine_123',
       'corine_124', 'corine_131', 'corine_132', 'corine_133', 'corine_141','corine_142', 'corine_211',
       'corine_212', 'corine_213', 'corine_221','corine_222', 'corine_223', 'corine_231', 'corine_241', 
       'corine_242','corine_243', 'corine_244', 'corine_311', 'corine_312', 'corine_313',
       'corine_321', 'corine_322', 'corine_323', 'corine_324', 'corine_331',
       'corine_332', 'corine_333', 'corine_334', 'corine_411', 'corine_412',
       'corine_421', 'corine_422', 'corine_511', 'corine_512', 'corine_521',
       'wkd_0', 'wkd_1', 'wkd_2', 'wkd_3', 'wkd_4', 'wkd_5', 'wkd_6',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_4',
       'frequency', 'f81', 'x', 'y']

In [6]:
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(df[train_cols], df[['fire']], test_size = 0.25, random_state = 42,shuffle=False)

In [37]:
from sklearn.ensemble import RandomForestClassifier# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 200, random_state = 42)# Train the model on training data
rf.fit(train_features, train_labels)

/apps/applications/python/anaconda3/5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [38]:
imp = pd.DataFrame([rf.feature_importances_], columns = train_cols).T
imp.columns = ['importance']
imp = imp.sort_values("importance", ascending = False)
imp

,importance
res_max,0.136147
rain_7days,0.093198
dom_vel,0.057867
f81,0.047662
x,0.045784
evi,0.043607
mean_temp,0.042706
y,0.042493
min_temp,0.040970
min_dew_temp,0.039062


In [ ]:
imp.to_csv('')

In [39]:
preds = rf.predict(test_features)
rf.score(test_features, test_labels)

0.72582602832097098

In [34]:
sum(preds == test_labels.values.flatten())/len(test_labels)

0.97666891436277814

Feature Importances old dataset new features

In [8]:
df = pd.read_csv('/work2/pa21/sgirtsou/production/datasets/randomnofire/old_random_new_feat_from_months.csv')

In [10]:
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(df[train_cols], df[['fire']], test_size = 0.25, random_state = 42,shuffle=False)

In [27]:
from sklearn.ensemble import RandomForestClassifier# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators=250, min_samples_split=180, min_samples_leaf=40, max_features=41, 
                            max_depth=20, criterion='entropy', class_weight={0: 1, 1: 9}, bootstrap=True)# Train the model on training data

rf.fit(train_features, train_labels)

/apps/applications/python/anaconda3/5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 9},
            criterion='entropy', max_depth=20, max_features=41,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=40,
            min_samples_split=180, min_weight_fraction_leaf=0.0,
            n_estimators=250, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [28]:
imp = pd.DataFrame([rf.feature_importances_], columns = train_cols).T
imp.columns = ['importance']
imp = imp.sort_values("importance", ascending = False)
imp

,importance
dom_vel,0.185974
ndvi_new,0.155583
evi,0.148117
lst_day,0.087270
f81,0.051748
mean_temp,0.049193
max_temp,0.047393
dem,0.037699
min_temp,0.035892
rain_7days,0.032476


In [35]:
imp.to_csv('/users/pa21/sgirtsou/production/feature_importances.csv')

In [32]:
test_features = test_features.fillna(0)

In [33]:
preds = rf.predict(test_features)
rf.score(test_features, test_labels)

0.75158939370445033

In [34]:
sum(preds == test_labels.values.flatten())/len(test_labels)

0.75158939370445033

# Μean decrease in impurity

In [39]:
import time
import numpy as np

start_time = time.time()
importances = rf.feature_importances_
std = np.std([
    rf.feature_importances_ for tree in rf.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: "
      f"{elapsed_time:.3f} seconds")

Elapsed time to compute the importances: 5.440 seconds


In [44]:
import pandas as pd
forest_importances = pd.Series(importances, index=train_cols)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [48]:
forest_importances.columns = ['feature','importance']
forest_importances = imp.sort_values("importance", ascending = False)

In [49]:
forest_importances

,importance
dom_vel,0.185974
ndvi_new,0.155583
evi,0.148117
lst_day,0.087270
f81,0.051748
mean_temp,0.049193
max_temp,0.047393
dem,0.037699
min_temp,0.035892
rain_7days,0.032476


# Permutation feature importance

In [50]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [55]:
df = df.fillna(0)

In [57]:
X_train, X_val, y_train, y_val = train_test_split(df[train_cols], df.fire, random_state=0)

In [58]:
model = Ridge(alpha=1e-2).fit(X_train, y_train)
model.score(X_val, y_val)

0.46806151611065255

In [61]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, X_val, y_val,
                          n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{diabetes.feature_names[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

ModuleNotFoundError: No module named 'sklearn.inspection'